In [26]:
from nltk.corpus import stopwords
import re
import pandas as pd
df = pd.read_csv('cuisine_data.csv')
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score

In [6]:
df.head()

,cuisine_description,cuisine
0,romaine lettuce black olives grape tomatoes ga...,greek
1,plain flour ground pepper salt tomatoes ground...,southern_us
2,eggs pepper salt mayonaise cooking oil green c...,filipino
3,water vegetable oil wheat salt,indian
4,black pepper shallots cornflour cayenne pepper...,indian


In [7]:
len(df.cuisine.value_counts())

20

In [8]:
df.cuisine.unique()

array(['greek', 'southern_us', 'filipino', 'indian', 'jamaican',
       'spanish', 'italian', 'mexican', 'chinese', 'british', 'thai',
       'vietnamese', 'cajun_creole', 'brazilian', 'french', 'japanese',
       'irish', 'korean', 'moroccan', 'russian'], dtype=object)

# Checking for NULLs and removing them

In [27]:
df.isnull().sum()

cuisine_description    0
cuisine                0
dtype: int64

# Dropping duplicates

In [28]:
df.shape

(39774, 2)

In [29]:
df.drop_duplicates(inplace=True)

In [30]:
df.shape

(39677, 2)

# Preprocessing the text

In [31]:
print(df['cuisine_description'].apply(lambda x: len(x.split(' '))).sum())

806112


In [32]:
special_character_remover = re.compile('[/(){}\[\]\|@,;]')
extra_symbol_remover = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

In [33]:
def clean_text(text):
    text = text.lower()
    text = special_character_remover.sub(' ', text)
    text = extra_symbol_remover.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text
    
df['cuisine_description'] = df['cuisine_description'].apply(clean_text)

In [35]:
print(df['cuisine_description'].apply(lambda x: len(x.split(' '))).sum())

803337


# Train Test split

In [37]:
from sklearn.model_selection import train_test_split
X = df.cuisine_description
y = df.cuisine
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [38]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((27773,), (11904,), (27773,), (11904,))

# Applying Logistic Regression

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

lr = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression()),
              ])

lr.fit(X_train,y_train)
y_pred1 = lr.predict(X_test)

print(f"Accuracy is : {accuracy_score(y_pred1,y_test)}")

/home/akshay/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy is : 0.7819220430107527


# Applying Naive Bayes Classifier

In [18]:
from sklearn.naive_bayes import MultinomialNB


naivebayes = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
naivebayes.fit(X_train, y_train)

y_pred = naivebayes.predict(X_test)

print(f'accuracy {accuracy_score(y_pred,y_test)}')


accuracy 0.6682627688172043


# Applying Xgboost Classifier

In [67]:
from xgboost import XGBClassifier

xgboost = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', XGBClassifier()),
              ])
xgboost.fit(X_train, y_train)

y_pred = xgboost.predict(X_test)

print(f'accuracy {accuracy_score(y_pred,y_test)}')


/home/akshay/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


accuracy 0.778309811827957


# Applying BERT

In [39]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer

In [40]:
from transformers import BertForSequenceClassification

In [41]:
from torch.utils.data import TensorDataset

In [42]:
possible_labels = df.cuisine.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'greek': 0,
 'southern_us': 1,
 'filipino': 2,
 'indian': 3,
 'jamaican': 4,
 'spanish': 5,
 'italian': 6,
 'mexican': 7,
 'chinese': 8,
 'british': 9,
 'thai': 10,
 'vietnamese': 11,
 'cajun_creole': 12,
 'brazilian': 13,
 'french': 14,
 'japanese': 15,
 'irish': 16,
 'korean': 17,
 'moroccan': 18,
 'russian': 19}

In [43]:
df['label'] = df.cuisine.replace(label_dict)

In [44]:
df.head()

,cuisine_description,cuisine,label
0,romaine lettuce black olives grape tomatoes ga...,greek,0
1,plain flour ground pepper salt tomatoes ground...,southern_us,1
2,eggs pepper salt mayonaise cooking oil green c...,filipino,2
3,water vegetable oil wheat salt,indian,3
4,black pepper shallots cornflour cayenne pepper...,indian,3


In [45]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['cuisine', 'label', 'data_type']).count()

cuisine_description
cuisine      label data_type                     
brazilian    13    train                      394
                   val                         69
british      9     train                      682
                   val                        121
cajun_creole 12    train                     1310
                   val                        231
chinese      8     train                     2266
                   val                        400
filipino     2     train                      642
                   val                        113
french       14    train                     2241
                   val                        396
greek        0     train                      996
                   val                        176
indian       3     train                     2547
                   val                        450
irish        16    train                      567
                   val                        100
italian      6     train                     6656
                   val                       1175
jamaican     4     train                      444
                   val                         78
japanese     15    train                     1204
                   val                        213
korean       17    train                      703
                   val                        124
mexican      7     train                     5465
                   val                        964
moroccan     18    train                      695
                   val                        123
russian      19    train                      416
                   val                         73
southern_us  1     train                     3654
                   val                        645
spanish      5     train                      839
                   val                        148
thai         10    train                     1306
                   val                        230
vietnamese   11    train                      698
                   val                        123

In [48]:
df.head()

,cuisine_description,cuisine,label,data_type
0,romaine lettuce black olives grape tomatoes ga...,greek,0,train
1,plain flour ground pepper salt tomatoes ground...,southern_us,1,train
2,eggs pepper salt mayonaise cooking oil green c...,filipino,2,train
3,water vegetable oil wheat salt,indian,3,train
4,black pepper shallots cornflour cayenne pepper...,indian,3,train


In [54]:
df.head()

,cuisine_description,cuisine,label,data_type
0,romaine lettuce black olives grape tomatoes ga...,greek,0,train
1,plain flour ground pepper salt tomatoes ground...,southern_us,1,train
2,eggs pepper salt mayonaise cooking oil green c...,filipino,2,train
3,water vegetable oil wheat salt,indian,3,train
4,black pepper shallots cornflour cayenne pepper...,indian,3,train


In [55]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [57]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].cuisine_description.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/akshay/anaconda3/envs/NLP/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [58]:
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].cuisine_description.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


In [59]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [60]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [61]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [62]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [63]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [66]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

KeyboardInterrupt: 